## Dependencies

In [ ]:
%pip install "giskard[llm]" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/

In [ ]:
%pip install langchain-groq -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.1 MB/s eta 0:00:00


In [ ]:
%pip install groq -q

## Package Imports

In [ ]:
import pandas as pd
from langchain.chains import LLMChain, SequentialChain
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

In [ ]:
import giskard
from giskard import Dataset, Model, scan

In [ ]:
from IPython.display import display, Markdown

## Groq LLM API Setup

In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('RED_GROQ_API_KEY')

In [ ]:
chat = ChatGroq(
    temperature=0.2,
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.2-1b-preview"
    )

In [ ]:
system = "You are a helpful assistant."
human = "{text}"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", human)
        ]
    )

chain = prompt | chat
response = chain.invoke({"text": "Tell me a scifi joke"})

print(response.content)

Here's one:

Why did the astronaut break up with his girlfriend before going to Mars?

Because he needed space.

I hope that one blasted off your funny bone!


## Newspaper Summary App

### Newpaper Articles Data

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv"
df = pd.read_csv(DATA_URL)
df.shape

(98401, 2)

In [ ]:
TEXT_COLUMN_NAME = "text"
RANDOM_STATE = 42

df_filtered = df[[TEXT_COLUMN_NAME]].sample(10, random_state=RANDOM_STATE, ignore_index=True)
df_filtered

,text
0,Students in Karnataka will get extra marks if ...
1,Syrian anti-aircraft defences on Monday shot d...
2,A Dinosaur-like creature's fossil was found du...
3,The Uttar Pradesh government is planning to fo...
4,Egyptian activist-actress Amal Fathy has been ...
5,World's richest person and Amazon CEO Jeff Bez...
6,Bangladesh cricket team's limited overs captai...
7,Mexican drug lord Joaquin 'El Chapo' Guzman ha...
8,Independent United Nations (UN) monitors have ...
9,After reports of three Amrapali Group companie...


In [ ]:
display(Markdown(df_filtered.iloc[1]['text']))

Syrian anti-aircraft defences on Monday shot down missiles over two air bases, Syria's state media said. The missiles targeted Shayrat air base in the Homs province and another base northeast of the capital Damascus. This comes days after the US, UK and France launched air strikes on Syrian chemical weapons facilities in retaliation for the alleged chemical attack in Douma.

In [ ]:
giskard_dataset = Dataset(df_filtered, target=None)

INFO:giskard.datasets.base:Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.


### Keyword Extraction + Summarization

In [ ]:
# First prompt to generate keywords related to the news article
KEYWORDS_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that generate a CSV list of \
    keywords related to a news article exerpt.\

    ----Example Format----\
    NEWS ARTICLE: news article here\
    KEYWORDS: keywords separated by commas here\

    Generate three to five important keywords for the news article.\
    """),

    ("human", """\
    NEWS ARTICLE: {text}\
    KEYWORDS:""")
    ])

In [ ]:
# Second chained prompt to generate a 1-line summary based on the given keywords from the first prompt
SUMMARY_PROMPT_TEMPLATE = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that generates a single line summary of the new article.\
    Limit to under 10 words.\

    ----Example Format----
    NEWS ARTICLE: product name here\
    KEYWORDS: keywords separated by commas here\
    SUMMARY: 1-line summary here\

    Make sure to answer in a consistent format.\
    """),

    ("human", """
    NEWS ARTICLE: {text}
    KEYWORDS: {keywords}
    SUMMARY: """)
    ])

In [ ]:
def generation_function(df: pd.DataFrame):
    llm = ChatGroq(temperature=0.2, groq_api_key=GROQ_API_KEY, model_name="llama-3.2-1b-preview")

    # Define the chains.
    keywords_chain = LLMChain(llm=llm, prompt=KEYWORDS_PROMPT_TEMPLATE, output_key="keywords")
    summary_chain = LLMChain(llm=llm, prompt=SUMMARY_PROMPT_TEMPLATE, output_key="summary")

    # Concatenate both chains.
    article_summary_chain = SequentialChain(chains=[keywords_chain, summary_chain],
                                                input_variables=["text"],
                                                output_variables=["summary"])

    return [article_summary_chain.invoke(article_text) for article_text in df['text']]

Chain Inference Test

In [ ]:
test_responses = generation_function(df_filtered.head(2))

In [ ]:
for res in test_responses:
  display(Markdown(res['text']))
  print('----------')
  display(Markdown(res['summary']))
  print("================================\n")

Students in Karnataka will get extra marks if their parents cast votes in the upcoming assembly elections, the Associated Management of Primary and Secondary Schools has announced. The "Encouraging Marks" will be added in the 2018-19 academic year. The association said, "After casting their votes, parents can visit member schools...and confirm that they voted by showing the indelible ink mark."

----------


Students in Karnataka to receive extra marks for parental voting.

Syrian anti-aircraft defences on Monday shot down missiles over two air bases, Syria's state media said. The missiles targeted Shayrat air base in the Homs province and another base northeast of the capital Damascus. This comes days after the US, UK and France launched air strikes on Syrian chemical weapons facilities in retaliation for the alleged chemical attack in Douma.

----------


Syria's air bases targeted by missiles.

## Giskard Model Wrapper

In [ ]:
# Wrap the description chain.
giskard_model = Model(
    model=generation_function,
    # A prediction function that encapsulates all the data pre-processing steps
    # and that could be executed with the dataset
    model_type="text_generation",
    # Either regression, classification or text_generation.
    name="News Article keywords and single line summary generator",  # Optional
    description="""Generate one line summary of news paper article based on \
    article exerpt and the associated keywords.""",
    # Is used to generate prompts
    feature_names=['text']  # Default: all columns of your dataset.
)

# Optional: Wrap a dataframe of sample input prompts to validate the model
# wrapping and to narrow specific tests' queries.
giskard_dataset = Dataset(df_filtered, target=None)

INFO:giskard.models.automodel:Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.
INFO:giskard.datasets.base:Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.


In [ ]:
giskard_dataset.df

,text
0,Students in Karnataka will get extra marks if ...
1,Syrian anti-aircraft defences on Monday shot d...
2,A Dinosaur-like creature's fossil was found du...
3,The Uttar Pradesh government is planning to fo...
4,Egyptian activist-actress Amal Fathy has been ...
5,World's richest person and Amazon CEO Jeff Bez...
6,Bangladesh cricket team's limited overs captai...
7,Mexican drug lord Joaquin 'El Chapo' Guzman ha...
8,Independent United Nations (UN) monitors have ...
9,After reports of three Amrapali Group companie...


In [ ]:
%%timeit

# Validate the wrapped model and dataset.
print(giskard_model.predict(giskard_dataset).prediction)

INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
INFO:giskard.utils.logging_utils:Predicted dataset with shape (10, 1) executed in 0:00:03.298043
INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
INFO:giskard.utils.logging_utils:Predicted dataset with shape (10, 1) executed in 0:00:00.016438
INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
INFO:giskard.utils.logging_utils:Predicted dataset with shape (10, 1) executed in 0:00:00.010706
INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
INFO:giskard.utils.logging_utils:Predicted dataset with shape (10, 1) executed in 0:00:00.010973
INFO:giskard.datasets.base:Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
INFO:giskard.utils.logging_utils:Predicted dataset with shape (10, 1) executed in 0:00:00.011334
INFO:giskard.dataset

[{'text': 'Students in Karnataka will get extra marks if their parents cast votes in the upcoming assembly elections, the Associated Management of Primary and Secondary Schools has announced. The "Encouraging Marks" will be added in the 2018-19 academic year. The association said, "After casting their votes, parents can visit member schools...and confirm that they voted by showing the indelible ink mark."', 'summary': 'Students in Karnataka to receive extra marks for parental voting.'}
 {'text': "Syrian anti-aircraft defences on Monday shot down missiles over two air bases, Syria's state media said. The missiles targeted Shayrat air base in the Homs province and another base northeast of the capital Damascus. This comes days after the US, UK and France launched air strikes on Syrian chemical weapons facilities in retaliation for the alleged chemical attack in Douma.", 'summary': "Syria's airbases targeted by missiles in retaliation for Douma attack."}
 {'text': "A Dinosaur-like creatur

## Giskard Red Team Scan

### Setup Groq LLM Client

In [ ]:
# def my_custom_llm(prompt):
#     # Implement the API call to Groq here
#     chat = ChatGroq(
#         temperature=0.2,
#         groq_api_key=GROQ_API_KEY,
#         model_name="llama-3.2-1b-preview"
#     )

#     response = call_groq_api(prompt)  # Replace with actual API call logic
#     return response

In [ ]:
def call_groq_api(prompt):
    """
    Calls the Groq API to generate a chat completion.
    """
    client = Groq(api_key=GROQ_API_KEY)  # Use your Groq API key

    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {"role": "user", "content": prompt}
            ],
            model="llama-3.2-1b-preview",  # Or another supported model
        )

        # Return the generated conten
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error during Groq API call: {e}")
        return None

In [ ]:
giskard.llm.custom_provider_map = [
    {"provider": "groq", "custom_handler": call_groq_api}
]

In [ ]:
giskard.llm.set_llm_model("groq/llama-3.2-1b-preview")

In [ ]:
from .groqClient import GroqClient

ImportError: attempted relative import with no known parent package

### Scan

In [ ]:
# Display options.
pd.set_option("display.max_colwidth", None)

In [ ]:
results = scan(giskard_model)

INFO:LiteLLM:
LiteLLM completion() model= llama-3.2-1b-preview; provider = groq



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers



AuthenticationError: litellm.AuthenticationError: AuthenticationError: GroqException - The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [ ]:
results = scan(giskard_model)

INFO:httpx:HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
display(results)

NameError: name 'results' is not defined